In [227]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sklm
import xgboost as xgb
import matplotlib
import lightgbm as lgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [138]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
submissions = pd.read_csv('SampleSubmission.csv')

In [139]:
print(train.shape)
print(test.shape)

(10001, 4)
(5177, 2)


In [84]:
traincols = list(train.columns)
testcols = list(test.columns)
#print(traincols)
#print(testcols)

In [6]:
train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [141]:
test = test.fillna('It was a good movie')
test.isna().sum()

tweet_id     0
safe_text    0
dtype: int64

In [86]:
y = np.array(train['label'])

In [124]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [87]:
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(train.safe_text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, test_size=0.2, shuffle=True)

In [118]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1,1), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [180]:
tfv.fit(x_train)
x_train_tfv =  tfv.transform(x_train1)
x_test_tfv = tfv.transform(x_test1)

#tfv.fit(list(test.safe_text.values))
#test_enc = tfv.transform(test.safe_text.values)

In [156]:
#test_enc = tfv.transform(test.safe_text.values)
pred = clf.predict(x_test_tfv)

print(xtrain_tfv.shape)
print(yvalid.shape)
print(xvalid_tfv.shape)

ValueError: X has 4047 features per sample; expecting 144730

In [160]:
from sklearn.metrics import mean_squared_error

clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfv, y)
predictions = clf.predict_proba(x_test_tfv)

#print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [131]:
test = test.fillna(0)
test.isna().sum()

tweet_id     0
safe_text    0
dtype: int64

In [137]:
train.safe_text.values

array(['Me &amp; The Big Homie meanboy3000 #MEANBOY #MB #MBS #MMR #STEGMANLIFE @ Stegman St. <url>',
       "I'm 100% thinking of devoting my career to proving autism isn't caused by vaccines due to the IDIOTIC posts I've seen about World Autism Day",
       '#whatcausesautism VACCINES, DO NOT VACCINATE YOUR CHILD', ...,
       'Interesting trends in child immunization in Oklahoma from <user> covering the <user> meeting.',
       'CDC Says Measles Are At Highest Levels In Decades: (<url> have returned in the U.S. to... <url>',
       'Pneumonia vaccine: for women w risk of pulmonary disease, see this ACOG tweet  <url>'],
      dtype=object)

In [134]:
train.dtypes

tweet_id      object
safe_text     object
label          int64
agreement    float64
dtype: object

In [135]:
test.dtypes

tweet_id     object
safe_text    object
dtype: object

In [165]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(x_train) + list(x_test))
xtrain_ctv =  ctv.transform(x_train) 
xtest_ctv = ctv.transform(x_test)

In [167]:
# Fitting a simple Logistic Regression on Counts
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, y)
predictions = clf.predict_proba(xtest_ctv)

#print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [147]:
combined = list(train.safe_text.values) + list(test.safe_text.values)
len(combined)

15178

In [148]:
len(test.safe_text.values)

5177

In [152]:
x_train = combined[:10001]
print(len(x_train))
x_test = combined[10001:]
print(len(x_test))

10001
5177


In [153]:
x_test[:2]

['<user> <user> ... &amp; 4 a vaccine given 2 healthy peeps, FDA think just not worth the AE risk unfortunately.',
 'Students starting school without whooping cough vaccinations <url> #scpick']

In [161]:
predictions

array([[0.33084543, 0.22957774, 0.43957683],
       [0.06058872, 0.2521447 , 0.68726658],
       [0.06616889, 0.78429222, 0.14953889],
       ...,
       [0.02631786, 0.66409337, 0.30958877],
       [0.08706373, 0.01135921, 0.90157706],
       [0.31217521, 0.20458462, 0.48324017]])

In [168]:
submissions.label = predictions
submissions.to_csv('NLPtest.csv', index = False)

In [223]:
lg = lgb.LGBMRegressor(boosting_type='gbdt', learning_rate=0.01, n_estimators=659, max_depth=50)
eval_set = [(x_test_tfv, y_test1)]
lg.fit(x_train_tfv, y_train1, eval_metric="rmse")

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.01, max_depth=50,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=659, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [210]:
xg_reg = xgb.XGBRegressor(objective = 'reg:squarederror', n_estimators = 3000, seed = 123, max_depth = 50,
                           learning_rate=0.01, booster = 'dart', base_score = 0.7, subsample = 0.8,
                           reg_lambda = 0.03)

eval_set = [(x_test_tfv, y_test1)]
xg_reg.fit(x_train_tfv, y_train1, eval_metric="rmse", eval_set=eval_set, verbose=True, early_stopping_rounds = 50)

[0]	validation_0-rmse:0.76350
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:0.76060
[2]	validation_0-rmse:0.75761
[3]	validation_0-rmse:0.75495
[4]	validation_0-rmse:0.75204
[5]	validation_0-rmse:0.74937
[6]	validation_0-rmse:0.74662
[7]	validation_0-rmse:0.74379
[8]	validation_0-rmse:0.74115
[9]	validation_0-rmse:0.73845
[10]	validation_0-rmse:0.73595
[11]	validation_0-rmse:0.73332
[12]	validation_0-rmse:0.73077
[13]	validation_0-rmse:0.72839
[14]	validation_0-rmse:0.72600
[15]	validation_0-rmse:0.72371
[16]	validation_0-rmse:0.72152
[17]	validation_0-rmse:0.71925
[18]	validation_0-rmse:0.71707
[19]	validation_0-rmse:0.71507
[20]	validation_0-rmse:0.71271
[21]	validation_0-rmse:0.71064
[22]	validation_0-rmse:0.70863
[23]	validation_0-rmse:0.70646
[24]	validation_0-rmse:0.70454
[25]	validation_0-rmse:0.70266
[26]	validation_0-rmse:0.70094
[27]	validation_0-rmse:0.69898
[28]	validation_0-rmse:0.69737
[29]	validation_0-rmse:0.69553
[30]	validation

[258]	validation_0-rmse:0.60256
[259]	validation_0-rmse:0.60253
[260]	validation_0-rmse:0.60249
[261]	validation_0-rmse:0.60243
[262]	validation_0-rmse:0.60239
[263]	validation_0-rmse:0.60237
[264]	validation_0-rmse:0.60231
[265]	validation_0-rmse:0.60226
[266]	validation_0-rmse:0.60224
[267]	validation_0-rmse:0.60218
[268]	validation_0-rmse:0.60216
[269]	validation_0-rmse:0.60211
[270]	validation_0-rmse:0.60209
[271]	validation_0-rmse:0.60209
[272]	validation_0-rmse:0.60202
[273]	validation_0-rmse:0.60198
[274]	validation_0-rmse:0.60196
[275]	validation_0-rmse:0.60192
[276]	validation_0-rmse:0.60191
[277]	validation_0-rmse:0.60184
[278]	validation_0-rmse:0.60177
[279]	validation_0-rmse:0.60174
[280]	validation_0-rmse:0.60173
[281]	validation_0-rmse:0.60167
[282]	validation_0-rmse:0.60164
[283]	validation_0-rmse:0.60164
[284]	validation_0-rmse:0.60159
[285]	validation_0-rmse:0.60157
[286]	validation_0-rmse:0.60154
[287]	validation_0-rmse:0.60149
[288]	validation_0-rmse:0.60149
[289]	va

XGBRegressor(base_score=0.7, booster='dart', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints=None,
       learning_rate=0.01, max_delta_step=0, max_depth=50,
       min_child_weight=1, missing=nan, monotone_constraints=None,
       n_estimators=3000, n_jobs=0, num_parallel_tree=1,
       objective='reg:squarederror', random_state=123, reg_alpha=0,
       reg_lambda=0.03, scale_pos_weight=1, seed=123, subsample=0.8,
       tree_method=None, validate_parameters=False, verbosity=None)

In [174]:
import numpy.random as nr
import sklearn.model_selection as ms
## Randomly sample cases to create independent training and test data
nr.seed(9988)
indx = range(np.array(x_train.shape[0]))
indx = ms.train_test_split(indx, test_size = 0.2)
x_train1 = x_train[indx[0]]
y_train1 = np.ravel(y[indx[0]])
x_test1 = x_train[indx[1]]
y_test1 = np.ravel(y[indx[1]])

In [183]:
print(x_train1.shape)
print(y_train1.shape)
print(x_test1.shape)
print(y_test1.shape)

(8000,)
(8000,)
(2001,)
(2001,)


In [172]:
x_train = np.array(x_train)

In [184]:
print(x_train_tfv.shape)
print(y_train1.shape)
print(x_test_tfv.shape)
print(y_test1.shape)

(8000, 4047)
(8000,)
(2001, 4047)
(2001,)


In [212]:
predictions = xg_reg.predict(x_test_tfv)
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test1, predictions) ** 0.5

0.6005280548011402

In [215]:
test_enc = tfv.transform(x_test)
pred = xg_reg.predict(test_enc)
submissions.label = pred
submissions.to_csv('NLPtestXG.csv', index = False)

In [224]:
from sklearn.model_selection import cross_val_score
cv_acc = cross_val_score(lg, x_train_tfv, y_train1, cv=5, scoring='neg_mean_absolute_error')
cv_acc

array([-0.42506279, -0.42125421, -0.43512611, -0.43309243, -0.42754693])

In [225]:
cv_acc.mean()

-0.4284164958066916

In [228]:
xgb_model = xgb.XGBRegressor()

parameters = {'learning_rate': [0.1, 0.05, 0.01], #so called `eta` value
              'max_depth': [15, 25, 35],
              'min_child_weight': [0.001, 0.01],
              'subsample': [0.8, 0.9, 1.0],
              'colsample_bytree': [0.9, 1.0],
              'n_estimators': [3000]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(x_train_tfv, n_folds=5, shuffle=True), 
                   scoring='rmse',
                   verbose=2, refit=True)

TypeError: __init__() got an unexpected keyword argument 'n_folds'

In [235]:
xgb_model = xgb.XGBRegressor()

parameters = {'learning_rate': [0.1, 0.05, 0.01], #so called `eta` value
              'max_depth': [15, 25, 35],
              'min_child_weight': [0.001, 0.01],
              'subsample': [0.8, 0.9, 1.0],
              'colsample_bytree': [0.9, 1.0],
              'n_estimators': [3000]}

clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(n_splits=5, shuffle=True), 
                   scoring='neg_mean_squared_error',
                   verbose=2, refit=True)

In [ ]:
clf.fit(x_train_tfv, y_train1)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


In [ ]:
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Neg MSE score', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))